### Fase 4: Creación de un Sistema de Recomendación
Diseña y desarrolla un sistema de recomendación basado en los datos recopilados.

**Modelos recomendados:**
- **Modelo colaborativo:** Basado en las preferencias de los usuarios.
- **Modelo basado en contenido:** Considerando géneros, directores y calificaciones.

El sistema debe permitir sugerir películas personalizadas para diferentes tipos de usuarios.

---

## Importamos librerías y configuraciones.

In [1]:
%load_ext autoreload
%autoreload 2

from config import *

---
---
## Creacion Sistema de Recomendacion

#### voy a realizar un sistema de recomendacion basado en contenido segun los argumentos de las peliculas.

#### posteriormente si me da tiempo, haré otro por popularidad segun Calificación, Director o Guionista.